In [1]:
from random import seed
from random import randrange
from random import random
from csv import reader
from math import exp
from sklearn.metrics import confusion_matrix
from sklearn.metrics import cohen_kappa_score
import numpy as np
import csv

In [2]:
# Define the Ada-Act activation function
def ada_act_activation(x, k0, k1):
    return k0 + k1 * x

# Load a CSV file
def loadCsv(filename):
    trainSet = []

    lines = csv.reader(open(filename, 'r'))
    dataset = list(lines)
    for i in range(len(dataset)):
        for j in range(4):
            dataset[i][j] = float(dataset[i][j])
        trainSet.append(dataset[i])
    return trainSet


In [3]:
# Rescale dataset columns to the range 0-1
def normalize(dataset, minmax):
    for row in dataset:
        for i in range(len(row) - 1):
            row[i] = (row[i] - minmax[i][0]) / (minmax[i][1] - minmax[i][0])

# Convert string column to float
def column_to_float(dataset, column):
    for row in dataset:
        try:
            row[column] = float(row[column])
        except ValueError:
            print("Error with row", column, ":", row[column])
            pass


In [4]:
# Convert string column to integer
def column_to_int(dataset, column):
    class_values = [row[column] for row in dataset]
    unique = set(class_values)
    lookup = dict()
    for i, value in enumerate(unique):
        lookup[value] = i
    for row in dataset:
        row[column] = lookup[row[column]]
    return lookup

# Split a dataset into k folds
def cross_validation_split(dataset, n_folds):
    dataset_split = list()
    dataset_copy = list(dataset)
    fold_size = int(len(dataset) / n_folds)
    for i in range(n_folds):
        fold = list()
        while len(fold) < fold_size:
            index = randrange(len(dataset_copy))
            fold.append(dataset_copy.pop(index))
        dataset_split.append(fold)
    return dataset_split

In [5]:
# Calculate accuracy percentage
def accuracy_met(actual, predicted):
    correct = 0
    for i in range(len(actual)):
        if actual[i] == predicted[i]:
            correct += 1
    return correct / float(len(actual)) * 100.0


In [6]:
# Evaluate an algorithm using a cross-validation split
def run_algorithm(dataset, algorithm, n_folds, *args):
    folds = cross_validation_split(dataset, n_folds)
    scores = list()
    for fold in folds:
        train_set = list(folds)
        train_set.remove(fold)
        train_set = sum(train_set, [])
        test_set = list()
        for row in fold:
            row_copy = list(row)
            test_set.append(row_copy)
            row_copy[-1] = None
        predicted = algorithm(train_set, test_set, *args)
        actual = [row[-1] for row in fold]
        accuracy = accuracy_met(actual, predicted)
        cm = confusion_matrix(actual, predicted)
        print('\n'.join([''.join(['{:4}'.format(item) for item in row]) for row in cm]))
        FP = cm.sum(axis=0) - np.diag(cm)
        FN = cm.sum(axis=1) - np.diag(cm)
        TP = np.diag(cm)
        TN = cm.sum() - (FP + FN + TP)
        TPR = TP / (TP + FN)
        TNR = TN / (TN + FP)
        Precision = TP / (TP + FP)
        Recall = TP / (TP + FN)
        F1 = 2 * Precision * Recall / (Precision + Recall)
        print("F1 Score:", F1)
        print("Mean F1 Score:", np.mean(F1))
        scores.append(accuracy)
    return scores


In [7]:
# Calculate neuron activation for an input using Ada-Act
def activate(weights, inputs, k0, k1):
    activation = weights[-1]
    for i in range(len(weights) - 1):
        activation += weights[i] * inputs[i]
    return ada_act_activation(activation, k0, k1)

# Transfer neuron activation using Ada-Act
def transfer(activation, k0, k1):
    return ada_act_activation(activation, k0, k1)

# Forward propagate input to a network output
def forward_propagate(network, row, k0, k1):
    inputs = row
    for layer in network:
        new_inputs = []
        for neuron in layer:
            activation = activate(neuron['weights'], inputs, k0, k1)
            neuron['output'] = transfer(activation, k0, k1)
            new_inputs.append(neuron['output'])
        inputs = new_inputs
    return inputs

In [8]:
# Calculate the derivative of a neuron output
def transfer_derivative(output, k1):
    return k1

# Backpropagate error and store it in neurons
def backward_propagate_error(network, expected, k1):
    for i in reversed(range(len(network))):
        layer = network[i]
        errors = list()
        if i != len(network) - 1:
            for j in range(len(layer)):
                error = 0.0
                for neuron in network[i + 1]:
                    error += (neuron['weights'][j] * neuron['delta'])
                errors.append(error)
        else:
            for j in range(len(layer)):
                neuron = layer[j]
                errors.append(expected[j] - neuron['output'])
        for j in range(len(layer)):
            neuron = layer[j]
            neuron['delta'] = errors[j] * transfer_derivative(neuron['output'], k1)


In [9]:
# Update network weights with error
def update_weights(network, row, l_rate, k0, k1):
    for i in range(len(network)):
        inputs = row[:-1]
        if i != 0:
            inputs = [neuron['output'] for neuron in network[i - 1]]
        for neuron in network[i]:
            for j in range(len(inputs)):
                neuron['weights'][j] += l_rate * neuron['delta'] * inputs[j]
            neuron['weights'][-1] += l_rate * neuron['delta']

# Train a network for a fixed number of epochs
def train_network(network, train, l_rate, n_epoch, n_outputs, k0, k1):
    for epoch in range(n_epoch):
        for row in train:
            outputs = forward_propagate(network, row, k0, k1)
            expected = [0 for i in range(n_outputs)]
            expected[int(row[-1])] = 1
            backward_propagate_error(network, expected, k1)
            update_weights(network, row, l_rate, k0, k1)


In [10]:
# Initialize a network with Ada-Act activation function
def initialize_network(n_inputs, n_hidden, n_outputs):
    network = list()
    hidden_layer = [{'weights': [random() for i in range(n_inputs + 1)],
                     'delta': [0 for i in range(n_inputs + 1)]} for i in range(n_hidden)]
    network.append(hidden_layer)
    output_layer = [{'weights': [random() for i in range(n_hidden + 1)],
                     'delta': [0 for i in range(n_hidden + 1)]} for i in range(n_outputs)]
    network.append(output_layer)
    return network

# Make a prediction with a network
def predict(network, row, k0, k1):
    outputs = forward_propagate(network, row, k0, k1)
    return outputs.index(max(outputs))


In [11]:
# Backpropagation Algorithm With Ada-Act Activation Function
def back_propagation(train, test, l_rate, n_epoch, n_hidden, k0, k1):
    n_inputs = len(train[0]) - 1
    n_outputs = len(set([row[-1] for row in train]))
    network = initialize_network(n_inputs, n_hidden, n_outputs)
    train_network(network, train, l_rate, n_epoch, n_outputs, k0, k1)
    predictions = []
    for row in test:
        prediction = predict(network, row, k0, k1)
        predictions.append(prediction)
    return predictions


In [13]:
# Test the backpropagation algorithm with Ada-Act activation function
seed(1)

# Load and prepare data
filename = 'IRIS.csv'

def loadCsv(filename):
    trainSet = []
    with open(filename, 'r') as file:
        lines = csv.reader(file)
        dataset = list(lines)
        headers = dataset[0]  # Get the column headers
        for row in dataset[1:]:  # Skip the first row (headers)
            for j in range(4):
                row[j] = float(row[j])
            trainSet.append(row)
    return trainSet

dataset = loadCsv(filename)

for i in range(len(dataset[0]) - 1):
    column_to_float(dataset, i)

# Convert class column to integers
lookup = column_to_int(dataset, len(dataset[0]) - 1)

# Find the min and max values for each column
def dataset_minmax(dataset):
    minmax = []
    for i in range(len(dataset[0])):
        column_values = [row[i] for row in dataset]
        column_min = min(column_values)
        column_max = max(column_values)
        minmax.append([column_min, column_max])
    return minmax

# Normalize input variables
minmax = dataset_minmax(dataset)
normalize(dataset, minmax)

# Evaluate algorithm
n_folds = 5
l_rate = 0.3
n_epoch = 500
n_hidden = 5
k0 = 0.2
k1 = 0.5

scores = run_algorithm(dataset, back_propagation, n_folds, l_rate, n_epoch, n_hidden, k0, k1)

print('Scores: %s' % scores)
print('Mean Accuracy: %.3f%%' % (sum(scores) / float(len(scores))))

  11   0   0
   0   7   1
   0   6   5
F1 Score: [1.         0.66666667 0.58823529]
Mean F1 Score: 0.7516339869281046
   9   1   0
   0   7   4
   0   3   6
F1 Score: [0.94736842 0.63636364 0.63157895]
Mean F1 Score: 0.7384370015948963
   6   0   0
   0   9   1
   0   2  12
F1 Score: [1.         0.85714286 0.88888889]
Mean F1 Score: 0.9153439153439153
  10   0   0
   0   9   1
   0   3   7
F1 Score: [1.         0.81818182 0.77777778]
Mean F1 Score: 0.8653198653198654
  13   0   0
   0   6   5
   0   1   5
F1 Score: [1.         0.66666667 0.625     ]
Mean F1 Score: 0.7638888888888888
Scores: [76.66666666666667, 73.33333333333333, 90.0, 86.66666666666667, 80.0]
Mean Accuracy: 81.333%
